In [32]:
import pandas as pd
import nltk

In [33]:
df = pd.read_csv("spotify_millsongdata.csv")

In [34]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [35]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [36]:
df.shape

(57650, 4)

In [37]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [38]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [39]:
df.head(10)

,artist,song,text
0,Dean Martin,I Feel Like A Feather In The Breeze,I feel like a feather in the breeze \r\nRidin...
1,Rod Stewart,Mama You Been On My Mind,Perhaps it's the color of the sun cut flat \r...
2,Gordon Lightfoot,Ribbon Of Darkness,Ribbon of darkness over me \r\nSince my true ...
3,Faces,Stay With Me,In the morning \r\nDon't say you love me \r\...
4,Queen,Liar,"I have sinned dear father, father I have sinne..."
5,Gino Vannelli,Nightwalker,I sleep at night \r\nOn a cloud high on the h...
6,Ariana Grande,Tattooed Heart,"You don't need a lot of money \r\nHoney, you ..."
7,Robbie Williams,Mess Me Up,From The Front \r\nTo The Back \r\nPut A Hum...
8,Neil Diamond,Don't Go Away Mad,I guess you think \r\nThat if you snap your f...
9,Manowar,Fast Taker,Mama is having problems \r\nalways comes to m...


In [40]:
df['text'][0]

"I feel like a feather in the breeze  \r\nRiding on high, high in the sky  \r\nHoney and I walking on air  \r\nSince the moment that she told me she cared  \r\nFeel like a feather in the breeze  \r\nFloating through space in your embrace  \r\nDancing on clouds way up above  \r\nSince the second that you beckoned my love  \r\nI'm happy  \r\nSo happy when you're near  \r\nMy troubles just disappear  \r\nAs soon as you're by my side well I'm satisfied  \r\nI feel like a feather in the breeze  \r\nHaving my fun, kissing the sun  \r\nAnd it's because you are the one  \r\nSo I'm singing like the birdies in the trees  \r\nAnd feeling like a feather in a breeze  \r\nI'm happy  \r\nSo happy when you're near  \r\nMy troubles just disappear  \r\nAs soon as you're by my side well I'm satisfied  \r\nI feel like a feather in the breeze  \r\nHaving my fun, kissing the sun  \r\nAnd it's because you are the one  \r\nSo I'm singing like the birdies in the trees  \r\nAnd feeling like a feather  \r\nFeeli

In [41]:
# df = df.sample(5000)

In [42]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [43]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [44]:
df = df.assign(feedback=0)

In [45]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [46]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [48]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [49]:
similarity[0]

array([1.00000000e+00, 4.43169401e-02, 2.57455164e-02, ...,
       2.06932010e-02, 9.22080723e-04, 5.24344004e-02])

In [50]:
df[df['song'] == df.iloc[0]['song']]

,artist,song,text,feedback
0,Dean Martin,I Feel Like A Feather In The Breeze,i feel like a feather in the breez ride on hig...,0


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np
from scipy.sparse import hstack, csr_matrix

In [52]:
df['combined_text'] = df['artist'] + ' ' + df['song'] + ' ' + df['text']
tfid_X = tfidvector.fit_transform(df['combined_text'])

df['feedback'] = np.random.choice([-1, 0, 1], size=len(df))

y = df['feedback']

X_train, X_test, y_train, y_test = train_test_split(tfid_X, y, test_size=0.2, random_state=42)

svc = SVC(probability=True)
svc.fit(X_train, y_train)

train_score = svc.score(X_train, y_train)
test_score = svc.score(X_test, y_test)

In [53]:
tfid_X.shape

(5000, 19914)

In [54]:
print("Training accuracy:", train_score)
print("Testing accuracy:", test_score)

Training accuracy: 0.97225
Testing accuracy: 0.314


In [55]:
ypred = svc.predict(X_test)

cm = confusion_matrix(y_test, ypred)
print(cm)

[[ 60 172  82]
 [ 59 165 107]
 [ 80 186  89]]


In [56]:
cr = classification_report(y_test, ypred)
print(cr)

              precision    recall  f1-score   support

          -1       0.30      0.19      0.23       314
           0       0.32      0.50      0.39       331
           1       0.32      0.25      0.28       355

    accuracy                           0.31      1000
   macro avg       0.31      0.31      0.30      1000
weighted avg       0.31      0.31      0.30      1000



In [57]:
def recommendation(song_df, svc_model):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    recommended_songs = []
    song_probs = []
    
    for m_id in distances[1:21]:
        artist = df.iloc[m_id[0]]['artist']
        song = df.iloc[m_id[0]]['song']
        text = df.iloc[m_id[0]]['text']
        
        combo = f"{artist} {song} {text}"
        X = tfidvector.fit_transform([combo])

        num_columns_to_add = tfid_X.shape[1] - X.shape[1]

        zeros_matrix = csr_matrix((X.shape[0], num_columns_to_add), dtype=np.float64)

        X = hstack([X, zeros_matrix])
    
        probability = svc_model.predict_proba(X)[0][1]
        song_probs.append(probability)
    
    sorted_indices = sorted(range(len(song_probs)), key=lambda i: song_probs[i], reverse=True)
    sorted_songs = [distances[i+1][0] for i in sorted_indices]
    
    recommended_songs = [df.iloc[i]['song'] for i in sorted_songs]
    
    return recommended_songs


In [58]:
recommendation(df.iloc[0]['song'], svc)

['A Touch Away',
 'Just Like That',
 'Skin Feeling',
 'I Know',
 'Just Like Me',
 'Feels Like Home',
 'I Feel Like Dying',
 'Both Sides Now',
 'Happy',
 'Can You Feel It',
 'Letting Go',
 'Somebody Knows',
 'Feelings',
 "Let's Make Sure We Kiss Goodbye",
 'I Like How It Feels',
 "I'm So High",
 'Fine As Fine Can Be',
 'In Another Land',
 'Happy',
 'Feels Like Home']

In [59]:
def update_model(return_info):
    for row in return_info:
        song_name = row[0]
        feedback_value = row[2]
        df.loc[df['song'] == song_name, 'feedback'] = feedback_value

    df['combined_text'] = df['artist'] + ' ' + df['song'] + ' ' + df['text']
    tfid_X = tfidvector.fit_transform(df['combined_text'])

    df['feedback'] = np.random.choice([-1, 0, 1], size=len(df))

    y = df['feedback']

    X_train, X_test, y_train, y_test = train_test_split(tfid_X, y, test_size=0.2, random_state=42)

    svc = SVC(probability=True)
    svc.fit(X_train, y_train)

    return svc

In [60]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))
pickle.dump(svc,open('svc.pkl','wb'))

In [61]:
df.head()

,artist,song,text,feedback,combined_text
0,Dean Martin,I Feel Like A Feather In The Breeze,i feel like a feather in the breez ride on hig...,0,Dean Martin I Feel Like A Feather In The Breez...
1,Rod Stewart,Mama You Been On My Mind,perhap it 's the color of the sun cut flat and...,0,Rod Stewart Mama You Been On My Mind perhap it...
2,Gordon Lightfoot,Ribbon Of Darkness,ribbon of dark over me sinc my true love walk ...,0,Gordon Lightfoot Ribbon Of Darkness ribbon of ...
3,Faces,Stay With Me,in the morn do n't say you love me 'caus i 'll...,1,Faces Stay With Me in the morn do n't say you ...
4,Queen,Liar,"i have sin dear father , father i have sin tri...",1,"Queen Liar i have sin dear father , father i h..."
